# Optimización de hiper-parámetros con Nevergrad

[Nevergrad](https://facebookresearch.github.io/nevergrad/index.html) es una librería de optimización numérica de código abierto desarrollada por **Facebook**. Aunque puede utilizarse para resolver cualquier problema de optimización, está diseñada para realizar una optimización de hiper-parámetros cuando estamos ajustando un modelo.

A diferencia de `GridSearch`, **Nevergrad** no realiza una búsqueda exhaustiva sobre todas las combinaciones posibles de valores, sino que utiliza metaheurísticas tales como los algoritmos evolutivos y sus variantes.

In [1]:
!pip install nevergrad

Un ejemplo rápido del funcionamiento de la librería sería optimizar una función bidimensional:

In [2]:
import nevergrad as ng

def square(x):
    return sum((x - 0.5) ** 2)

# optimization on x as an array of shape (2,)
optimizer = ng.optimizers.NGOpt(parametrization=2, budget=100)
recommendation = optimizer.minimize(square)  # best value
print(recommendation.value)

(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=<module 'time' (built-in)>, Sun Jun  6 15:11:08 2021)
[0.50000003 0.49999993]


Vamos a construir un modelo de red neuronal para ver cómo podemos optimizar los hiperparámetros para obtener el mejor ajuste del modelo. Utilizaremos un conjunto de datos sencillo para no dilatar mucho el tiempo de cómputo de la optimización de los hiperparámetros: **Fashion MNIST**.

In [50]:
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.datasets import fashion_mnist
import matplotlib.pyplot as plt

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

num_train_images = X_train.shape[0]
num_test_images = X_test.shape[0]
image_height = X_train.shape[1]
image_width = X_train.shape [2]
X_train = X_train.reshape(X_train.shape[0], image_height*image_width)
X_test = X_test.reshape(X_test.shape[0], image_height*image_width)
X_train = X_train/255.0
X_test = X_test/255.0
y_train = K.utils.to_categorical(y_train, 10)
y_test = K.utils.to_categorical(y_test, 10)

Para que funcione **Nevergrad** necesitamos una función de evaluación para medir la calidad de un conjunto determinado de hiper-parámetros. Para ello encapsulamos la construcción de la red neuronal junto con la evaluación de la misma:

In [55]:
def network(d_1, d_2, dr, if_dense3, d_3, act='relu', x_train=X_train, y_train=y_train, x_test=X_test, y_test=y_test):
    model = K.Sequential()
    model.add(K.layers.Dense(d_1, activation=act, input_shape=(784,)))
    model.add(K.layers.Dropout(dr))
    model.add(K.layers.Dense(d_2, activation=act))
    if if_dense3: # la tercera capa es opcional
        model.add(K.layers.Dense(d_3, activation=act))
    model.add(K.layers.Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=256,
                        epochs=10,
                        verbose=0,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    return float(score[0])    # el valor de 'fitness' será el valor de evaluación de la función de coste

Como podemos observar, los hiper-parámetros de la red son los siguientes:

- `d_1`: número de neuronas de la primera capa densa.
- `d_2`: número de neuronas de la segunda capa densa.
- `dr`: tasa de *Dropout* entre la primera y segunda capa.
- `if_dense3`: ¿añadimos tercera capa?
- `d_3`: número de neuronas de la tercera capa densa (opcional).

**Nevergrad** denomina **intrumentalización** a la definición de los hiper-parámetros. Observa que los nombres de los parámetros de la instrumentalización coinciden con los parámetros de la función de evaluación. También es necesario elegir uno de los [optimizadores](https://facebookresearch.github.io/nevergrad/optimizers_ref.html#optimizers) que vienen incluidos en **Nevergrad**.

Por último, bastará con ejecutar la meta-optimización consistente en minimizar la función de evaluación que hemos definido anteriormente.

In [62]:
import nevergrad as ng

d_1 = ng.p.TransitionChoice(range(10, 60, 10))
d_2 = ng.p.Choice(range(10, 60, 10))
dr = ng.p.Scalar(lower=0.2, upper=0.7)
if_dense3 = ng.p.Choice([True, False])
d_3 = ng.p.Choice(range(10, 60, 10))
#act = ng.p.Choice(['sigmoid', 'relu'])

params = ng.p.Instrumentation(d_1, d_2, dr, if_dense3, d_3)
optimizer = ng.optimizers.TwoPointsDE(parametrization=params, budget=10000)
best = optimizer.minimize(network, batch_mode=False)

Test loss: 0.41100960969924927
Test accuracy: 0.8508999943733215
Test loss: 0.4905564486980438
Test accuracy: 0.8219000101089478
Test loss: 0.8029636740684509
Test accuracy: 0.680899977684021
Test loss: 0.3949854373931885
Test accuracy: 0.8555999994277954
Test loss: 0.8093854188919067
Test accuracy: 0.676800012588501
Test loss: 0.5914907455444336
Test accuracy: 0.7678999900817871
Test loss: 0.41467413306236267
Test accuracy: 0.8550999760627747
Test loss: 0.4008086621761322
Test accuracy: 0.8578000068664551
Test loss: 0.43379488587379456
Test accuracy: 0.8472999930381775
Test loss: 0.68550705909729
Test accuracy: 0.7294999957084656
Test loss: 0.427870512008667
Test accuracy: 0.8425999879837036
Test loss: 0.6401051878929138
Test accuracy: 0.7678999900817871
Test loss: 0.3965599238872528
Test accuracy: 0.858299970626831
Test loss: 0.42326122522354126
Test accuracy: 0.8483999967575073
Test loss: 0.7196761965751648
Test accuracy: 0.7835000157356262
Test loss: 0.3942267596721649
Test accurac

In [64]:
best.value

((50, 50, 0.26367643514925654, True, 50), {})

---

Creado por **Raúl Lara Cabrera** (raul.lara@upm.es)

<img src="https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png">